In [1]:
import pandas as pd
import requests
import json
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as ss
import statsmodels.api as sm
import pylab as py
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import pingouin as pg
import statsmodels.formula.api as smf

In [2]:
cars = pd.read_csv('/home/jupyter-o-demchenko-18/statistic/Lesson7/cars.csv')

In [3]:
cars.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [4]:
# проверка пропущенных значений (их сумма)
cars.isna().sum()

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

3. Сгенерируем первый признак.

Использовать полное название машины – не самый хороший вариант, поэтому создадим новый признак – марку автомобиля (company). Для этого используйте столбец CarName, разбейте значения ячеек по пробелу и запишите в колонку первый элемент. Например:

'audi 100 ls' → 'audi'
Сколько всего уникальных марок машины встречается в датасете? Столбец CarName с полным названием машины удалите из датасета, а также car_ID, они не пригодятся для дальнейшего анализа.

In [5]:
# разделяем строки в столбце CarName по пробелу и с помощью функции expand=True создаем новый датафрейм new_df, состоящий из разделенных строк
new_df = cars['CarName'].str.split(' ', expand=True)
new_df

,0,1,2,3,4
0,alfa-romero,giulia,None,None,None
1,alfa-romero,stelvio,None,None,None
2,alfa-romero,Quadrifoglio,None,None,None
3,audi,100,ls,None,None
4,audi,100ls,None,None,None
...,...,...,...,...,...
200,volvo,145e,(sw),None,None
201,volvo,144ea,None,None,None
202,volvo,244dl,None,None,None
203,volvo,246,None,None,None


In [6]:
# переименуем колонки
new_df.columns=['brand_cars','1','2', '3', '4']
new_df

,brand_cars,1,2,3,4
0,alfa-romero,giulia,None,None,None
1,alfa-romero,stelvio,None,None,None
2,alfa-romero,Quadrifoglio,None,None,None
3,audi,100,ls,None,None
4,audi,100ls,None,None,None
...,...,...,...,...,...
200,volvo,145e,(sw),None,None
201,volvo,144ea,None,None,None
202,volvo,244dl,None,None,None
203,volvo,246,None,None,None


In [7]:
# соединяем новый датафрейм и исходный по столбцу (axis=1)
final_df = pd.concat([cars,new_df],axis=1)
final_df

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,horsepower,peakrpm,citympg,highwaympg,price,brand_cars,1,2,3,4
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,111,5000,21,27,13495.0,alfa-romero,giulia,None,None,None
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,111,5000,21,27,16500.0,alfa-romero,stelvio,None,None,None
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,154,5000,19,26,16500.0,alfa-romero,Quadrifoglio,None,None,None
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,102,5500,24,30,13950.0,audi,100,ls,None,None
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,115,5500,18,22,17450.0,audi,100ls,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,201,-1,volvo 145e (sw),gas,std,four,sedan,rwd,front,109.1,...,114,5400,23,28,16845.0,volvo,145e,(sw),None,None
201,202,-1,volvo 144ea,gas,turbo,four,sedan,rwd,front,109.1,...,160,5300,19,25,19045.0,volvo,144ea,None,None,None
202,203,-1,volvo 244dl,gas,std,four,sedan,rwd,front,109.1,...,134,5500,18,23,21485.0,volvo,244dl,None,None,None
203,204,-1,volvo 246,diesel,turbo,four,sedan,rwd,front,109.1,...,106,4800,26,27,22470.0,volvo,246,None,None,None


In [8]:
# удаляем ненужные колонки по очереди
final_df = final_df.drop('car_ID',axis=1)
final_df

,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,...,horsepower,peakrpm,citympg,highwaympg,price,brand_cars,1,2,3,4
0,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,168.8,...,111,5000,21,27,13495.0,alfa-romero,giulia,None,None,None
1,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,168.8,...,111,5000,21,27,16500.0,alfa-romero,stelvio,None,None,None
2,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,171.2,...,154,5000,19,26,16500.0,alfa-romero,Quadrifoglio,None,None,None
3,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,176.6,...,102,5500,24,30,13950.0,audi,100,ls,None,None
4,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,176.6,...,115,5500,18,22,17450.0,audi,100ls,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1,volvo 145e (sw),gas,std,four,sedan,rwd,front,109.1,188.8,...,114,5400,23,28,16845.0,volvo,145e,(sw),None,None
201,-1,volvo 144ea,gas,turbo,four,sedan,rwd,front,109.1,188.8,...,160,5300,19,25,19045.0,volvo,144ea,None,None,None
202,-1,volvo 244dl,gas,std,four,sedan,rwd,front,109.1,188.8,...,134,5500,18,23,21485.0,volvo,244dl,None,None,None
203,-1,volvo 246,diesel,turbo,four,sedan,rwd,front,109.1,188.8,...,106,4800,26,27,22470.0,volvo,246,None,None,None


In [9]:
# подсчет уникальных значений марок машин
final_df['brand_cars'].nunique()

28

4. Теперь внимательнее посмотрите на уникальные значения company. Часть из них оказалась с ошибками!

In [10]:
# выведим уникальные значения
final_df['brand_cars'].unique()

array(['alfa-romero', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'maxda', 'mazda', 'buick', 'mercury',
       'mitsubishi', 'Nissan', 'nissan', 'peugeot', 'plymouth', 'porsche',
       'porcshce', 'renault', 'saab', 'subaru', 'toyota', 'toyouta',
       'vokswagen', 'volkswagen', 'vw', 'volvo'], dtype=object)

In [11]:
# замена строчных букв на прописные
final_df['brand_cars'] = final_df['brand_cars'].str.lower()

In [12]:
# замена неправильных названий
final_df['brand_cars'] = final_df['brand_cars'].str.replace('maxda', 'mazda', regex=True)
final_df['brand_cars'] = final_df['brand_cars'].str.replace('porcshce', 'porsche', regex=True)
final_df['brand_cars'] = final_df['brand_cars'].str.replace('toyouta', 'toyota', regex=True)
final_df['brand_cars'] = final_df['brand_cars'].str.replace('vokswagen', 'volkswagen', regex=True)
final_df['brand_cars'] = final_df['brand_cars'].str.replace('vw', 'volkswagen', regex=True)


/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs
/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.4.0, the latest is 0.5.0.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


In [13]:
# подсчет уникальных значений
final_df['brand_cars'].nunique()

22

In [14]:
final_df.head()

,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,...,horsepower,peakrpm,citympg,highwaympg,price,brand_cars,1,2,3,4
0,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,168.8,...,111,5000,21,27,13495.0,alfa-romero,giulia,None,None,None
1,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,168.8,...,111,5000,21,27,16500.0,alfa-romero,stelvio,None,None,None
2,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,171.2,...,154,5000,19,26,16500.0,alfa-romero,Quadrifoglio,None,None,None
3,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,176.6,...,102,5500,24,30,13950.0,audi,100,ls,None,None
4,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,176.6,...,115,5500,18,22,17450.0,audi,100ls,None,None,None


5. Чтобы не перегружать модель большим количеством предикторов, оставим только часть из них:

'company', 'fueltype', 'aspiration','carbody', 'drivewheel', 'wheelbase', 'carlength','carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower'

In [15]:
# вывод названий колонок датафрейма
for column in final_df:
   print(column)

symboling
CarName
fueltype
aspiration
doornumber
carbody
drivewheel
enginelocation
wheelbase
carlength
carwidth
carheight
curbweight
enginetype
cylindernumber
enginesize
fuelsystem
boreratio
stroke
compressionratio
horsepower
peakrpm
citympg
highwaympg
price
brand_cars
1
2
3
4


In [16]:
# вывод нового датафрейма с необходимыми колонками
df = final_df[['brand_cars', 'fueltype', 'aspiration','carbody', 'drivewheel', 'wheelbase', 'carlength','carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower', 'price']]

In [17]:
# рассчет корреляции относительно колонки price и сортировка сверху вниз (опционально)
df_corr = df.corr()['price'].sort_values(ascending=False).round(2)

In [18]:
df_corr

price         1.00
enginesize    0.87
curbweight    0.84
horsepower    0.81
carwidth      0.76
carlength     0.68
wheelbase     0.58
boreratio     0.55
Name: price, dtype: float64

6. Последний шаг в подготовке данных: линейная регрессия в python не справляется с категориальными переменными (типом object в pandas), поэтому давайте применим pd.get_dummies(). Пример использования:

df_dummy = pd.get_dummies(data=cars[[список_столбцов_типа_object]], drop_first = True)
Не забудьте присоединить к полученному датасету столбцы с переменными других типов :)

Сколько колонок теперь имеется в датасете?

In [19]:
# проверим типы данных в df
df.dtypes

brand_cars         object
fueltype           object
aspiration         object
carbody            object
drivewheel         object
wheelbase         float64
carlength         float64
carwidth          float64
curbweight          int64
enginetype         object
cylindernumber     object
enginesize          int64
boreratio         float64
horsepower          int64
price             float64
dtype: object

In [20]:
# перевод категориальных переменных типа object в dummy
df_dummy = pd.get_dummies(data=df[['brand_cars', 'fueltype', 'aspiration', 'carbody', 'drivewheel', 'enginetype', 'cylindernumber']], drop_first = True)
df_dummy

,brand_cars_audi,brand_cars_bmw,brand_cars_buick,brand_cars_chevrolet,brand_cars_dodge,brand_cars_honda,brand_cars_isuzu,brand_cars_jaguar,brand_cars_mazda,brand_cars_mercury,...,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_rotor,cylindernumber_five,cylindernumber_four,cylindernumber_six,cylindernumber_three,cylindernumber_twelve,cylindernumber_two
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
201,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
202,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
203,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0


In [21]:
# соединение нового датафрейма со старым
df_dummy_finish = pd.concat([df, df_dummy], axis=1)
df_dummy_finish

,brand_cars,fueltype,aspiration,carbody,drivewheel,wheelbase,carlength,carwidth,curbweight,enginetype,...,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_rotor,cylindernumber_five,cylindernumber_four,cylindernumber_six,cylindernumber_three,cylindernumber_twelve,cylindernumber_two
0,alfa-romero,gas,std,convertible,rwd,88.6,168.8,64.1,2548,dohc,...,0,0,0,0,0,1,0,0,0,0
1,alfa-romero,gas,std,convertible,rwd,88.6,168.8,64.1,2548,dohc,...,0,0,0,0,0,1,0,0,0,0
2,alfa-romero,gas,std,hatchback,rwd,94.5,171.2,65.5,2823,ohcv,...,0,0,1,0,0,0,1,0,0,0
3,audi,gas,std,sedan,fwd,99.8,176.6,66.2,2337,ohc,...,1,0,0,0,0,1,0,0,0,0
4,audi,gas,std,sedan,4wd,99.4,176.6,66.4,2824,ohc,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,volvo,gas,std,sedan,rwd,109.1,188.8,68.9,2952,ohc,...,1,0,0,0,0,1,0,0,0,0
201,volvo,gas,turbo,sedan,rwd,109.1,188.8,68.8,3049,ohc,...,1,0,0,0,0,1,0,0,0,0
202,volvo,gas,std,sedan,rwd,109.1,188.8,68.9,3012,ohcv,...,0,0,1,0,0,0,1,0,0,0
203,volvo,diesel,turbo,sedan,rwd,109.1,188.8,68.9,3217,ohc,...,1,0,0,0,0,0,1,0,0,0


In [22]:
df_dummy_finish.shape
# число новых колонок 56, но 7 колонок повторяются, поэтому итоговое кол-во колонок 56-7=49

(205, 56)

7. Сначала построим небольшую модель всего с одним предиктором цены (price) – horsepower.

Какой процент изменчивости объясняет полученная модель? (округлите до целого)

In [47]:
price_horsepower_model = smf.ols(formula='price ~ horsepower', data=df_dummy_finish).fit().summary()
price_horsepower_model


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     382.2
Date:                Mon, 31 Jan 2022   Prob (F-statistic):           1.48e-48
Time:                        07:23:04   Log-Likelihood:                -2024.0
No. Observations:                 205   AIC:                             4052.
Df Residuals:                     203   BIC:                             4059.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3721.7615    929.849     -4.003      0.000   -5555.163   -1888.360
horsepower   163.2631      8.351     19.549      0.000     146.796     179.730
==============================================================================
Omnibus:                       47.741   Durbin-Watson:                   0.792
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               91.702
Skew:                           1.141   Prob(JB):                     1.22e-20
Kurtosis:                       5.352   Cond. No.                         314.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

8. Теперь – две модели:

-модель со всеми предикторами

-модель со всеми предикторами, кроме марок машин

Обратите внимание на изменения в R**2, коэффициентах и их значимости. Какую модель лучше оставить? 

In [44]:
results_all = smf.ols('price ~ brand_cars + fueltype + aspiration+carbody+drivewheel+wheelbase+carlength+carwidth+curbweight+enginetype+cylindernumber+enginesize+boreratio+horsepower', df_dummy_finish).fit()
print(results_all.summary())
print("R2_all: ", results_all.rsquared)

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     81.09
Date:                Mon, 31 Jan 2022   Prob (F-statistic):           4.86e-89
Time:                        07:21:11   Log-Likelihood:                -1804.2
No. Observations:                 205   AIC:                             3702.
Df Residuals:                     158   BIC:                             3858.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

In [45]:
# модель без брендов машин
results_without_brand_car = smf.ols('price ~ fueltype + aspiration+carbody+drivewheel+wheelbase+carlength+carwidth+curbweight+enginetype+cylindernumber+enginesize+boreratio+horsepower', df_dummy_finish).fit()
print(results_without_brand_car.summary())
print("R2_without_brand_car: ", results_without_brand_car.rsquared)

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.901
Method:                 Least Squares   F-statistic:                     72.32
Date:                Mon, 31 Jan 2022   Prob (F-statistic):           9.86e-81
Time:                        07:21:31   Log-Likelihood:                -1881.6
No. Observations:                 205   AIC:                             3817.
Df Residuals:                     178   BIC:                             3907.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

In [48]:
# Вывод: оставляем модель без брендов машин(т.к. коэфф R**2 не сильно изменился после выброса предиктора марка, и некоторые бренды имеют значение p<0.05)